In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Flipkart/flipkart_com-ecommerce_sample_1050.csv")

In [3]:
df["category_name"] = df["product_category_tree"].str.split(r">>", regex=False, expand=True)[0]
df["category_name"] = df["category_name"].str.replace('["', '')

In [4]:
df['category_name'].value_counts()

,count
category_name,
Home Furnishing,150
Baby Care,150
Watches,150
Home Decor & Festive Needs,150
Kitchen & Dining,150
Beauty and Personal Care,150
Computers,150


*****************************************************

**Ajout des images dans des sous dossiers spécifiques à leur catégorie**

*****************************************************


In [ ]:


import os
import shutil

# Chemin vers le répertoire contenant les images
base_dir = '/content/drive/MyDrive/Flipkart/Images'

# Créer les sous-dossiers de catégories s'ils n'existent pas
for category in df['category_name'].unique():
    os.makedirs(os.path.join(base_dir, category), exist_ok=True)

# Déplacer chaque image dans le dossier de sa catégorie
for _, row in df.iterrows():
    src_path = os.path.join(base_dir, row['image'])  # chemin d'origine de l'image
    dest_path = os.path.join(base_dir, row['category_name'], row['image'])  # chemin de destination
    if os.path.exists(src_path):  # vérifier si le fichier existe pour éviter les erreurs
        shutil.move(src_path, dest_path)
    else:
        print(f"Image non trouvée : {src_path}")

print("Images regroupées par catégorie.")


In [6]:
from sklearn.model_selection import train_test_split

descriptions = df["description"].tolist()
categories = df["category_name"].tolist()

# Division des données
train_texts, val_texts, train_labels, val_labels = train_test_split(
    descriptions, categories, test_size=0.2, random_state=42
)

In [7]:
from transformers import DistilBertTokenizer, TFDistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

model = TFDistilBertModel.from_pretrained("distilbert-base-uncased")

def tokenize_texts(texts):
    return tokenizer(
        texts, padding=True, truncation=True, return_tensors="tf"
    )

train_encodings = tokenize_texts(train_texts)
val_encodings = tokenize_texts(val_texts)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistil

In [8]:
import tensorflow as tf

input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="attention_mask")

embeddings = model(input_ids, attention_mask=attention_mask)[0]
output = tf.keras.layers.GlobalAveragePooling1D()(embeddings)
output = tf.keras.layers.Dense(len(set(categories)), activation="softmax")(output)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

train_labels = tf.convert_to_tensor(train_labels)
val_labels = tf.convert_to_tensor(val_labels)


In [9]:
from sklearn.preprocessing import LabelEncoder

# Encodage des catégories
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)

train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
val_labels = tf.convert_to_tensor(val_labels, dtype=tf.int32)

model.fit(
    {"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"]},
    train_labels,
    validation_data=(
        {"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"]},
        val_labels
    ),
    epochs=3,
    batch_size=16
)


Epoch 1/3


FailedPreconditionError: Graph execution error:

Detected at node Adam/StatefulPartitionedCall_101 defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/loc

**IMAGE CLASSIFICATION**

In [ ]:
IMG_SIZE = (224, 224)  # Pour EfficientNet, utiliser (299, 299) pour InceptionV3
BATCH_SIZE = 32

# Chargement des images depuis le répertoire avec partition en ensembles d’entraînement et de validation
train_dataset = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/Flipkart/Images',
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    '/content/drive/MyDrive/Flipkart/Images',
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
])

# Normalisation pour correspondre à l'entraînement ImageNet
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(data_augmentation(x)), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
from tensorflow.keras.applications import EfficientNetB0  # Remplacer par InceptionV3 si nécessaire
from tensorflow.keras import layers, Model

# Chargement du modèle pré-entraîné sans la dernière couche de classification
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Congélation des poids du modèle de base pour un fine-tuning minimal
base_model.trainable = False

# Ajout des nouvelles couches
x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dropout(0.2)(x)  # Regularisation pour éviter l'overfitting
output = layers.Dense(train_generator.num_classes, activation='softmax')(x)  # Nombre de classes

# Construction du modèle
model = Model(inputs=base_model.input, outputs=output)

# Compilation du modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
